In [58]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0,
    groq_api_key="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"
)
response = llm.invoke("The first person to land on moon was .")
print(response.content)

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [77]:

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45029?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Lead Software Engineer

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologyLead Software EngineerBeaverton, OregonBecome a Part of the NIKE, Inc. TeamNIKE, Inc. does mor

In [78]:

from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
print(res.content)


Here is the extracted job posting in JSON format:

```
{
  "role": "Lead Software Engineer",
  "experience": "6+ years software engineering experience",
  "skills": [
    "Java",
    "Golang",
    "Python",
    "Node.js",
    "React",
    "JavaScript",
    "Cloud Native technologies on AWS",
    "Kubernetes",
    "Terraform",
    "Ansible"
  ],
  "description": "We are looking for a Lead Software Engineer to build and develop technology for Nike’s future in direct partnership with engineering leaders across the enterprise. This role collaborates with Engineers, Architects, Product Managers, and Program Managers to create reusable, well-architected and highly scalable solutions to modernize Nike’s technology landscape."
}
```


In [79]:
from langchain_core.output_parsers import JsonOutputParser

json_parser=JsonOutputParser()
json_res=json_parser.parse(res.content)
json_res

{'role': 'Lead Software Engineer',
 'experience': '6+ years software engineering experience',
 'skills': ['Java',
  'Golang',
  'Python',
  'Node.js',
  'React',
  'JavaScript',
  'Cloud Native technologies on AWS',
  'Kubernetes',
  'Terraform',
  'Ansible'],
 'description': 'We are looking for a Lead Software Engineer to build and develop technology for Nike’s future in direct partnership with engineering leaders across the enterprise. This role collaborates with Engineers, Architects, Product Managers, and Program Managers to create reusable, well-architected and highly scalable solutions to modernize Nike’s technology landscape.'}

In [80]:
import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [81]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [87]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}],
 [{'links': 'https://example.com/full-stack-js-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backen

In [85]:
job = json_res
job['skills']

['Java',
 'Golang',
 'Python',
 'Node.js',
 'React',
 'JavaScript',
 'Cloud Native technologies on AWS',
 'Kubernetes',
 'Terraform',
 'Ansible']

In [88]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}
    
    ### INSTRUCTION:
    You are Hitanshu, a student passionate about technology, AI, and software development. You are reaching out to a potential client to 
    introduce yourself and express your interest in contributing to their project. 
    In the email, highlight your skills, your eagerness to learn, and how your knowledge could be applied to help with the job described 
    above. Mention any relevant projects or skills you have that would make you a strong candidate for the role.
    Be professional, concise, and make sure to demonstrate your enthusiasm for the opportunity.
    Also, feel free to add links or references to any of your work, portfolio, or skills that might be relevant: {link_list}
    Remember, you are Hitanshu, a student with a strong interest in this field, and you’re eager to offer your help.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)


chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expressed Interest in Lead Software Engineer Role at Nike

Dear Hiring Manager,

I am thrilled to express my interest in the Lead Software Engineer role at Nike, as described in the job description. As a student passionate about technology, AI, and software development, I am excited about the opportunity to contribute to Nike's future technology landscape.

With a strong foundation in programming languages such as Java, Python, and JavaScript, I am confident in my ability to develop scalable and well-architected solutions. My experience with Cloud Native technologies on AWS, Kubernetes, Terraform, and Ansible aligns with the job requirements, and I am eager to apply my knowledge to modernize Nike's technology landscape.

In my previous projects, I have demonstrated my skills in building reusable and scalable solutions using React, Node.js, and Golang. I have also worked on various projects that showcase my proficiency in full-stack development, DevOps, and machine learning. Yo